In [98]:
import pandas as pd

In [99]:
data = pd.read_csv("Customer-Churn-Records.csv")

In [100]:
def round_1(number, multiple_of = 10):
    number = round(number/multiple_of)
    number *= multiple_of
    return int(number)

balance_func = lambda x: round_1(x, 10000)
data['Balance'] = data['Balance'].map(balance_func)

salary_func = lambda x: round_1(x, 1000)
data['EstimatedSalary'] = data['EstimatedSalary'].map(salary_func)

credit_func = lambda x: round_1(x, 10)
data['CreditScore'] = data['CreditScore'].map(credit_func)

point_func = lambda x: round_1(x, 100)
data['Point Earned'] = data['Point Earned'].map(point_func)

age_func = lambda x: round_1(x, 5)
data['Age'] = data['Age'].map(age_func)

data = data.drop(columns = ['RowNumber', 'CustomerId', 'Surname', 'Geography'])

In [101]:
data

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,cardtype,gender,geography
0,620,40,2,0,1,1,1,101000,1,1,2,500,4,1,1
1,610,40,1,80000,1,0,1,113000,0,1,3,500,4,1,2
2,500,40,8,160000,3,1,0,114000,1,1,3,400,4,1,1
3,700,40,1,0,2,0,0,94000,0,0,5,400,2,1,1
4,850,45,2,130000,1,1,1,79000,0,0,5,400,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,770,40,5,0,2,1,0,96000,0,0,1,300,4,0,1
9996,520,35,10,60000,1,1,1,102000,0,0,5,800,3,0,1
9997,710,35,7,0,1,0,1,42000,1,1,3,600,1,1,1
9998,770,40,3,80000,2,1,0,93000,1,1,2,300,2,0,3


In [102]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

In [103]:
lookup = dict()
names = ['Tenure', 'NumOfProducts', 'HasCrCard', 'Complain', 'IsActiveMember', 'Satisfaction Score', 'cardtype', 'gender', 'geography', 'Balance', 'EstimatedSalary', 'CreditScore', 'Point Earned', 'Age'] 
for name in names:
    lookup[name] = (train.groupby([name, 'Exited']).size()).div(len(train)).div(train.groupby(name).size().div(len(train)), axis=0, level=0)

In [104]:
exited_prob = data.groupby('Exited').size().div(len(data))

In [105]:
exited_prob

Exited
0    0.7962
1    0.2038
dtype: float64

In [106]:
def predict(input):

    prob_exit = 1
    prob_not_exit = 1
    for feature in input:
        try:
            prob_exit = prob_exit * lookup[feature][input[feature]][1]
        except:
            prob_exit = prob_exit * (1/len(train))
        
        try:
            prob_not_exit = prob_not_exit * lookup[feature][input[feature]][0]
        except:
            prob_not_exit = prob_not_exit * (1/len(train))


        prob_exit = prob_exit*exited_prob[1]
        prob_not_exit = prob_not_exit*exited_prob[0]

        prob_exit = prob_exit/(prob_exit + prob_not_exit)

        prob_not_exit = 1 - prob_exit
        
        if prob_exit > prob_not_exit:
            return 1
        else:
            return 0

In [107]:
L = list(test.itertuples(index=False, name=None))

In [108]:
truth = []
pred = []
for l in L:
    truth.append(l[8])
    input = {'Tenure': l[2],
                'NumOfProducts': l[4],
                'HasCrCard': l[5],
                'Complain': l[9],
                'IsActiveMember': l[6], 
                'Satisfaction Score': l[10], 
                'cardtype': l[-3], 
                'gender': l[-2], 
                'geography': l[-1], 
                'Balance': l[3], 
                'EstimatedSalary': l[7], 
                'CreditScore': l[0], 
                'Point Earned': l[-4], 
                'Age':l[1]}

    pred.append(predict(input))

In [109]:
from sklearn.metrics import classification_report
print(classification_report(truth, pred))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89      1597
           1       0.00      0.00      0.00       403

    accuracy                           0.80      2000
   macro avg       0.40      0.50      0.44      2000
weighted avg       0.64      0.80      0.71      2000



c:\Users\toant\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\toant\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\toant\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
